In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPool2D, BatchNormalization, LeakyReLU
import numpy as np
from PIL import Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import TensorBoard
import time


In [2]:
NAME = "single_rgb_image_big_dataset_{}".format(int(time.time()))


In [3]:
tensorboard = TensorBoard(log_dir="logs/{}".format(NAME))

In [4]:
# def create_data_via_ImageDataGenerator():
#     train = ImageDataGenerator(rescale=1/255)
#     validation = ImageDataGenerator(rescale=1/255)
#     train_dataset = train.flow_from_directory("../Data/Train/smallTown",
#                                               target_size=(60, 60), batch_size=3, class_mode='binary')
#     validation_dataset = train.flow_from_directory("../Data/Train/smallTown",
#                                                target_size=(60, 60), batch_size=3, class_mode='binary')


In [5]:
def load_data_via_numpy_load():
    return (np.load("CNN_trainset_big_singleRGB_shuffled_X.npy")/255, np.load("CNN_trainset_big_singleRGB_shuffled_Y.npy"))


In [6]:
X, Y = load_data_via_numpy_load()


In [7]:
X


array([[[[[0.03137255],
          [0.04705882],
          [0.04705882]],

         [[0.03529412],
          [0.05098039],
          [0.05098039]],

         [[0.03529412],
          [0.05490196],
          [0.05098039]],

         ...,

         [[0.01960784],
          [0.03921569],
          [0.03921569]],

         [[0.01960784],
          [0.03529412],
          [0.03921569]],

         [[0.01960784],
          [0.03921569],
          [0.03921569]]],


        [[[0.03137255],
          [0.04705882],
          [0.04705882]],

         [[0.03137255],
          [0.05098039],
          [0.05098039]],

         [[0.03529412],
          [0.05098039],
          [0.05098039]],

         ...,

         [[0.02352941],
          [0.04705882],
          [0.05098039]],

         [[0.02745098],
          [0.04705882],
          [0.05098039]],

         [[0.01960784],
          [0.03921569],
          [0.04313725]]],


        [[[0.03137255],
          [0.04705882],
          [0.04705882]],

    

In [8]:
X.shape

(79776, 60, 60, 3, 1)

In [9]:


X = X.reshape(-1, 60, 60, 3)

X.shape


(79776, 60, 60, 3)

In [10]:
def build_model():
    model = Sequential()

    # 1st Conv-Layer
    model.add(Conv2D(128, (3, 3), input_shape=(X.shape[1:])))
    model.add(Activation(LeakyReLU(alpha=0.2)))
    model.add(BatchNormalization())
    model.add(MaxPool2D(2, 2))

    # 2nd Conv-Layer
    model.add(Conv2D(192, (3, 3)))
    model.add(Activation(LeakyReLU(alpha=0.2)))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))
    
    # 3rd Conv-Layer
    model.add(Conv2D(320, (3, 3)))
    model.add(Activation(LeakyReLU(alpha=0.2)))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))

    # 4th Conv-Layer
    model.add(Conv2D(576, (3, 3)))
    model.add(Activation(LeakyReLU(alpha=0.2)))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))

    model.add(Flatten())

    # 1st Fully-Connected
    model.add(Dense(4096))
    model.add(Activation(LeakyReLU(alpha=0.2)))

    # 2nd Fully-Connected
    model.add(Dense(4096))
    model.add(Activation(LeakyReLU(alpha=0.2)))

    # 3rd Fully-Connected
    model.add(Dense(1))
    model.add(Activation("sigmoid"))
    return model


model = build_model()


In [11]:

model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 58, 58, 128)       3584      
_________________________________________________________________
activation (Activation)      (None, 58, 58, 128)       0         
_________________________________________________________________
batch_normalization (BatchNo (None, 58, 58, 128)       512       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 29, 29, 128)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 27, 27, 192)       221376    
_________________________________________________________________
activation_1 (Activation)    (None, 27, 27, 192)       0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 27, 27, 192)       7

In [12]:
model.fit(X, Y, batch_size=32, epochs=50, validation_split=0.1, callbacks=[tensorboard])

Epoch 1/50
2244/2244 [==============================] - 159s 53ms/step - loss: 0.3540 - accuracy: 0.9725 - val_loss: 0.0019 - val_accuracy: 0.9999
Epoch 2/50
2244/2244 [==============================] - 112s 50ms/step - loss: 0.2038 - accuracy: 0.9911 - val_loss: 6.5411e-04 - val_accuracy: 0.9999
Epoch 3/50
2244/2244 [==============================] - 111s 50ms/step - loss: 0.9281 - accuracy: 0.9885 - val_loss: 0.0128 - val_accuracy: 0.9999
Epoch 4/50
2244/2244 [==============================] - 109s 49ms/step - loss: 0.0994 - accuracy: 0.9959 - val_loss: 0.0785 - val_accuracy: 0.9887
Epoch 5/50
2244/2244 [==============================] - 111s 50ms/step - loss: 0.0371 - accuracy: 0.9973 - val_loss: 0.0068 - val_accuracy: 0.9986
Epoch 6/50
2244/2244 [==============================] - 111s 49ms/step - loss: 0.3090 - accuracy: 0.9939 - val_loss: 1.6429e-07 - val_accuracy: 1.0000
Epoch 7/50
2244/2244 [==============================] - 110s 49ms/step - loss: 0.0139 - accuracy: 0.9987 - val

# Test


In [13]:
# model = tf.keras.models.load_model("./models/single_rgb_image_big_dataset")


In [14]:
# # Alias

# img1 = Image.open(
#     "D:\Main\MA_PROGR\Data\Train\LED_Wand_Aufnahmen\Alias\LED_Wand_20001.png")
# img1 = np.asarray(img1)/255




# img1 = img1.reshape(1,60,60,3)
# print(img1.shape)
# # print(img1)
# res1 = model.predict(img1)
# print(res1)


In [15]:
# # NoAlias

# img2 = Image.open(
#     "D:\\Main\\MA_PROGR\\Data\\Train\\LED_Wand_Aufnahmen\\NoAlias\\abstract_cross_203.png")
# img2 = np.asarray(img2)/255

# img2 = img2.reshape(1, 60, 60, 3)
# img2.shape
# # print(img2)
# res2 = model.predict(img2)
# print(res2)


In [16]:
# import matplotlib.pyplot as plt
# plt.imshow(img2.reshape(60, 60, 3))


In [17]:
tf.keras.models.save_model(model, "./models/{}".format(NAME))

INFO:tensorflow:Assets written to: ./models/single_rgb_image_big_dataset_1652967042\assets


INFO:tensorflow:Assets written to: ./models/single_rgb_image_big_dataset_1652967042\assets
